# nbgrader workflow

Let op: voor een correcte werking moet deze workflow in een eigen map uitgevoerd worden. Dit notebook moet in deze map staan en Jupyter moet vanuit deze map opgestart te worden. Dit is een beperking van nbgrader.

In [81]:
import os
import re
from glob import glob
import pandas as pd

from IPython.display import display, Markdown, Javascript

display(Javascript("if(!(window.location.pathname.indexOf('notebooks') == 1 && window.location.pathname.split('/').length == 3)) { alert('De workflow-notebook staat niet in de root-map van Jupyter!'); }"))

<IPython.core.display.Javascript object>

## 0. Configuratie

In [30]:
course_id = "netwerkanalyse"
assignment_id = "tentamen1"

## 1. nbgrader configureren
code-stub overschrijven etc.

In [31]:
config = '''
c = get_config()

c.Exchange.course_id = "%s"
c.ClearSolutions.code_stub = {'python': '# JOUW CODE HIER', 'markdown': '....'}
''' % course_id

with open('nbgrader_config.py', 'w') as f:
    f.write(config)

## 2. Assignment-map aanmaken

In [32]:
!nbgrader db assignment add {assignment_id}
!mkdir -p source/{assignment_id}

!ls -l

# Met curl template ophalen naar source/assignment_id/assignment_id.ipynb

display(Markdown('---\n## 3. Assignment maken\nHet notebook-template staat in **source/%s/%s.ipynb**  \n<a class="btn btn-primary" style="margin-top: 10px; text-decoration: none;" href="/notebooks/source/%s/%s.ipynb" target="_blank">Open notebook</a>' % (assignment_id, assignment_id, assignment_id, assignment_id)))

[DbAssignmentAddApp | INFO] Creating/updating assignment with ID 'tentamen1': {'duedate': None}
total 296
drwxr-xr-x@ 14 Jesse  staff     448 Apr  3 13:46 Voorbeeld
-rw-r--r--@  1 Jesse  staff  135168 Apr  3 15:47 gradebook.db
-rw-r--r--@  1 Jesse  staff     139 Apr  3 15:47 nbgrader_config.py
-rw-r--r--@  1 Jesse  staff   11103 Apr  3 15:47 nbgrader_workflow.ipynb
drwxr-xr-x   3 Jesse  staff      96 Apr  3 15:47 source


---
## 3. Assignment maken
Het notebook-template staat in **source/tentamen1/tentamen1.ipynb**  
<a class="btn btn-primary" style="margin-top: 10px; text-decoration: none;" href="/notebooks/source/tentamen1/tentamen1.ipynb" target="_blank">Open notebook</a>

nbgrader documentatie: http://nbgrader.readthedocs.io/en/stable/user_guide/creating_and_grading_assignments.html

## 4. Maak een studentversie

In [33]:
!rm -rf release
!nbgrader assign {assignment_id} --create

student_nb = glob('release/%s/*.ipynb' % assignment_id)[0]
student_nb_filename = os.path.basename(student_nb)
student_nb_name = os.path.splitext(student_nb_filename)[0]

display(Markdown('---\nDe studentversie bevindt zich in **%s**  \n<a class="btn btn-primary" style="margin-top: 10px; text-decoration: none;" href="/notebooks/%s" target="_blank">Open notebook</a>' % (student_nb, student_nb)))

[AssignApp | INFO] Updating/creating assignment 'tentamen1': {}
[AssignApp | INFO] Converting notebook /Users/Jesse/Dropbox/Notebook-exams/notebook-exams/Scripts/nbgrader/source/./tentamen1/tentamen1.ipynb
[AssignApp | INFO] Writing 963 bytes to /Users/Jesse/Dropbox/Notebook-exams/notebook-exams/Scripts/nbgrader/release/./tentamen1/tentamen1.ipynb
[AssignApp | INFO] Setting destination file permissions to 644


---
De studentversie bevindt zich in **release/tentamen1/tentamen1.ipynb**  
<a class="btn btn-primary" style="margin-top: 10px; text-decoration: none;" href="/notebooks/release/tentamen1/tentamen1.ipynb" target="_blank">Open notebook</a>

## 5. Verzamel inzendingen

In [36]:
submissions_zip = 'submissions.zip'

!unzip -j {submissions_zip} -d "original-submissions"

nbs_copied = 0

for f in glob('original-submissions/*.ipynb'):
    nb = os.path.basename(f)
    
    ids = re.findall(r'\d{5,}', nb)
    if len(ids):
        student_id = ids[0]
        
        !mkdir -p submitted/{student_id}/{assignment_id}
        !cp {f} submitted/{student_id}/{assignment_id}/{student_nb_filename}
        
        nbs_copied += 1

display(Markdown('---\n%d notebook(s) gevonden en gekopiëerd  \n<a class="btn btn-primary" style="margin-top: 10px; text-decoration: none;" href="/tree/submitted" target="_blank">Open inzendingen-map</a>' % nbs_copied))

unzip:  cannot find or open submissions.zip, submissions.zip.zip or submissions.zip.ZIP.


---
0 notebook(s) gevonden en gekopiëerd  
<a class="btn btn-primary" style="margin-top: 10px; text-decoration: none;" href="/tree/submitted" target="_blank">Open inzendingen-map</a>

## 6. Autograden

In [37]:
!nbgrader autograde {assignment_id} --create

display(Markdown('---\n<a class="btn btn-primary" style="margin-top: 10px; text-decoration: none;" href="/formgrader/gradebook/%s/%s" target="_blank">Open manual grading</a>' % (assignment_id, student_nb_name)))

[AutogradeApp | INFO] Creating/updating student with ID '10735291': {}
[AutogradeApp | INFO] Overwriting files with master versions from the source directory
[AutogradeApp | INFO] Sanitizing /Users/Jesse/Dropbox/Notebook-exams/notebook-exams/Scripts/nbgrader/submitted/10735291/tentamen1/tentamen1.ipynb
[AutogradeApp | INFO] Converting notebook /Users/Jesse/Dropbox/Notebook-exams/notebook-exams/Scripts/nbgrader/submitted/10735291/tentamen1/tentamen1.ipynb
[AutogradeApp | INFO] Writing 963 bytes to /Users/Jesse/Dropbox/Notebook-exams/notebook-exams/Scripts/nbgrader/autograded/10735291/tentamen1/tentamen1.ipynb
[AutogradeApp | INFO] Autograding /Users/Jesse/Dropbox/Notebook-exams/notebook-exams/Scripts/nbgrader/autograded/10735291/tentamen1/tentamen1.ipynb
[AutogradeApp | INFO] Converting notebook /Users/Jesse/Dropbox/Notebook-exams/notebook-exams/Scripts/nbgrader/autograded/10735291/tentamen1/tentamen1.ipynb
[AutogradeApp | INFO] Executing notebook with kernel: python3
[AutogradeApp | IN

---
<a class="btn btn-primary" style="margin-top: 10px; text-decoration: none;" href="/formgrader/gradebook/tentamen1/tentamen1" target="_blank">Open manual grading</a>

## 7. students-answers.csv genereren

In [55]:
import numpy as np

q_oud = '''
    SELECT
        submitted_assignment.student_id,
        grade_cell.name AS question_name,
        grade_cell.max_score,
        grade.needs_manual_grade AS needs_grading,
        grade.auto_score,
        grade.manual_score,
        grade.extra_credit,
        comment.auto_comment,
        comment.manual_comment
    FROM grade
        INNER JOIN submitted_notebook ON submitted_notebook.id = grade.notebook_id
        INNER JOIN submitted_assignment ON submitted_assignment.id = submitted_notebook.assignment_id
        INNER JOIN grade_cell ON grade_cell.id = grade.cell_id
        INNER JOIN solution_cell ON solution_cell.name = grade_cell.name AND solution_cell.notebook_id = grade_cell.notebook_id
        LEFT JOIN comment ON comment.notebook_id = grade.notebook_id AND comment.cell_id = solution_cell.id
'''

q = '''
    SELECT
        submitted_assignment.student_id,
        grade_cell.name AS question_name,
        grade_cell.max_score,
        grade.needs_manual_grade AS needs_grading,
        grade.auto_score,
        grade.manual_score,
        grade.extra_credit
    FROM grade
        INNER JOIN submitted_notebook ON submitted_notebook.id = grade.notebook_id
        INNER JOIN submitted_assignment ON submitted_assignment.id = submitted_notebook.assignment_id
        INNER JOIN grade_cell ON grade_cell.id = grade.cell_id
'''

df = pd.read_sql_query(q, 'sqlite:///gradebook.db')
df = df.set_index(['student_id', 'question_name'])
df['final_score'] = np.where(~pd.isnull(df['manual_score']), df['manual_score'], df['auto_score']) + df['extra_credit'].fillna(0)

df.to_csv('student-results.csv')

df

,,max_score,needs_grading,auto_score,manual_score,extra_credit,final_score
student_id,question_name,,,,,,
10735291,vraag1,3.0,0,0.0,3.0,None,3.0


## 8. Analyseren

In [57]:
per_student = df.groupby('student_id')
grades = per_student['final_score'].sum() / per_student['max_score'].sum() * 10

grades

student_id
10735291    10.0
dtype: float64